In [52]:
#Imports
import pandas as pd

In [53]:
#Lecture du fichier csv brut
df = pd.read_csv("https://drive.switch.ch/index.php/s/CqLVP8sXoLSxvYz/download")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
#Nbre de lignes/colonnes
df.shape

(20864, 106)

In [55]:
#Aperçu
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,...,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,21974,https://www.airbnb.com/rooms/21974,20200613063451,2020-06-15,Comf. double room/private bathroom,It is private double room with private bathroo...,Comfortable double room with private bathroom....,It is private double room with private bathroo...,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/deea733e-e...,NaN,73163,https://www.airbnb.com/users/show/73163,Andres,2010-01-24,"Barcelona, Catalonia, Spain","Hello I am a Professional designer, a traveler...",within a few hours,100%,91%,f,https://a0.muscache.com/im/pictures/user/9cbe7...,https://a0.muscache.com/im/pictures/user/9cbe7...,El Gòtic,3.0,3.0,"['email', 'phone', 'reviews', 'manual_offline'...",t,t,"Barcelona, CT, Spain",El Gòtic,el Barri Gòtic,...,$0.00,3,90,3,3,90,90,3.0,90.0,3 weeks ago,t,16,16,16,88,2020-06-15,2,0,2017-05-16,2017-11-06,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,HUTB-120440,NaN,f,f,moderate,f,f,3,1,2,0,0.05
1,23197,https://www.airbnb.com/rooms/23197,20200613063451,2020-06-16,Forum DeLuxe 5 mins Walk To CCIB Center & Sea★,I do not accept groups of young people under 2...,Elegant spacious apartment suitable for 6. Amp...,I do not accept groups of young people under 2...,none,Strategically located in the area of Parc del ...,"As a Superhost, I aim to make your stay as com...","Tram T4 Campus Diagonal- Besos, over the ro...",You book the entire apartment for yourselves.,"Please do contact Tony, your check-in manager,...","As a Superhost, I aim to make your stay as com...",NaN,NaN,https://a0.muscache.com/im/pictures/738532/806...,NaN,90417,https://www.airbnb.com/users/show/90417,Etain (Marnie),2010-03-09,"Catalonia, Spain","Hi. I'm Etain, though everyone calls me by my...",within an hour,100%,100%,t,https://a0.muscache.com/im/users/90417/profile...,https://a0.muscache.com/im/users/90417/profile...,El Besòs i el Maresme,6.0,6.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Sant Adria de Besos, Barcelona, Spain",Sant Martí,el Besòs i el Maresme,...,$10.00,3,30,3,7,1125,1125,3.2,1125.0,5 weeks ago,t,30,57,83,322,2020-06-16,52,10,2011-03-15,2019-12-15,95.0,10.0,10.0,10.0,10.0,9.0,9.0,t,HUTB-005057,NaN,t,f,strict_14_with_grace_period,f,t,2,2,0,0,0.46
2,34241,https://www.airbnb.com/rooms/34241,20200613063451,2020-06-15,Private Double room - Plaza Real,NaN,It is a Private double room with private bathr...,It is a Private double room with private bathr...,none,NaN,NaN,NaN,NaN,NaN,late check in fee might apply,NaN,NaN,https://a0.muscache.com/im/pictures/2437facc-2...,NaN,73163,https://www.airbnb.com/users/show/7316

In [56]:
#Nombre d'annonces selon le nombre minimum de nuits
df["minimum_minimum_nights"].value_counts().head(20)

1      6489
2      4892
3      3149
32     2324
30     1058
31      846
4       666
5       429
7       218
60       94
6        85
90       82
15       66
33       54
28       51
10       48
20       34
35       32
14       26
180      23
Name: minimum_minimum_nights, dtype: int64

In [57]:
#Suppression des annonces avec un minimum exigé de plus de 7 nuits
df = df[df["minimum_minimum_nights"] <= 7]

In [58]:
#Sélection des dimensions d'intérêt
df_subset = df[["id", "neighbourhood_cleansed", "room_type", "accommodates", "guests_included", "price", "cleaning_fee"]]

In [59]:
#Renommage des colonnes
df_final = df_subset.rename(columns = {"neighbourhood_cleansed" : "quartier", "room_type" : "type_chambre", "accommodates" : "occupation_max", "guests_included" : "hotes_incluses", "price" : "prix", "cleaning_fee" : "frais_nettoyage"})

In [60]:
#Supression du caractère "$" dans les indications de prix
df_final["prix"] = df_final["prix"].str[1:]
df_final["frais_nettoyage"] = df_final["frais_nettoyage"].str[1:]

In [61]:
#Aperçu
df_final.head()

,id,quartier,type_chambre,occupation_max,hotes_incluses,prix,frais_nettoyage
0,21974,el Barri Gòtic,Private room,2,2,80.00,20.00
1,23197,el Besòs i el Maresme,Entire home/apt,6,3,220.00,80.00
2,34241,el Barri Gòtic,Private room,2,1,100.00,40.00
3,34981,el Barri Gòtic,Entire home/apt,9,4,227.00,67.00
4,35379,l'Antiga Esquerra de l'Eixample,Private room,2,1,40.00,6.00


In [62]:
#Suppression du caractères "," dans ées indications de prix
df_final["prix"] = df_final["prix"].str.replace(',', '')
df_final["frais_nettoyage"] = df_final["frais_nettoyage"].str.replace(',', '')

In [63]:
#Conversion des prix en float
df_final["prix"] = pd.to_numeric(df_final["prix"]).astype(float)
df_final["frais_nettoyage"] = pd.to_numeric(df_final["frais_nettoyage"]).astype(float)

In [64]:
#Calcul du prix total (prix indiqué + frais de ménage)
df_final["prix_total_pp"] = (df_final["prix"] + df_final["frais_nettoyage"]) / df_final["hotes_incluses"]

In [65]:
#Affichage du tableau trié selon le prix total
df_final.sort_values(by = ["prix_total_pp"], ascending=False).head()

,id,quartier,type_chambre,occupation_max,hotes_incluses,prix,frais_nettoyage,prix_total_pp
10848,27802058,Sant Antoni,Hotel room,1,1,6000.0,0.0,6000.0
10852,27802471,Sant Antoni,Hotel room,1,1,6000.0,0.0,6000.0
10847,27802009,Sant Antoni,Hotel room,2,1,6000.0,0.0,6000.0
10885,27916731,Sant Antoni,Hotel room,2,1,6000.0,0.0,6000.0
10905,27951003,l'Antiga Esquerra de l'Eixample,Hotel room,1,1,6000.0,0.0,6000.0


In [66]:
#Aperçu du nombre de valeurs NaN par colonne
df_final.isna().sum()

id                    0
quartier              0
type_chambre          0
occupation_max        0
hotes_incluses        0
prix                  0
frais_nettoyage    3466
prix_total_pp      3466
dtype: int64

In [67]:
#Suppression des entrées pour lesquells les frais de nettoyage ne sont pas renseignées
df_final = df_final.dropna(subset = ["frais_nettoyage"])

In [68]:
#Calcul du prix moyen par quartier
df_final["prix_moyen"] = df_final.groupby("quartier")["prix_total_pp"].transform("mean")

In [69]:
#Calcul du prix minimum par quartier
df_final["prix_min"] = df_final.groupby("quartier")["prix_total_pp"].transform("min")

In [70]:
#Calcul du prix maximum par quartier
df_final["prix_max"] = df_final.groupby("quartier")["prix_total_pp"].transform("max")

In [71]:
#Calcul du prix médian par quartier
df_final["prix_med"] = df_final.groupby("quartier")["prix_total_pp"].transform("median")

In [72]:
#Calcul du nombre d'annonces par quartier
df_final["nb_annonces"] = df_final.groupby("quartier")["prix_total_pp"].transform("count")

In [73]:
#Calcul du nombre d'annonces selon le type de logement par quartier
types_compteur = df_final.groupby(["quartier", "type_chambre"])["id"].count()

In [74]:
#Affichage de la variable ci-dessus
types_compteur

quartier         type_chambre   
Baró de Viver    Entire home/apt     1
Can Baró         Entire home/apt    16
                 Private room        9
Canyelles        Entire home/apt     1
                 Private room        1
                                    ..
les Roquetes     Entire home/apt     1
                 Private room       13
                 Shared room         1
les Tres Torres  Entire home/apt    14
                 Private room        8
Name: id, Length: 182, dtype: int64

In [75]:
#Insertion de colonnes et initialisation à 0
df_final["nb_log_entier"] = 0
df_final["nb_chambre_privee"] = 0
df_final["nb_chambre_hotel"] = 0
df_final["nb_chambre_ptg"] = 0

In [76]:
#Création de la liste des quartiers
liste_quartiers = df_final["quartier"].unique()

In [77]:
#Pour chacun des quartier,, mise à jour du nombre de logement par type (chambre partagée, privée, ...)
for quartier in liste_quartiers :
  try :
    df_final.loc[df_final.quartier == quartier, "nb_log_entier"] = types_compteur[quartier]["Entire home/apt"]
  except :
    #Cas où il n'existe pas d'annonce correspondant au type de logement pour ce quartier
    pass
  
  try :
    df_final.loc[df_final.quartier == quartier, "nb_chambre_privee"] = types_compteur[quartier]["Private room"]
  except :
    #Cas où il n'existe pas d'annonce correspondant au type de logement pour ce quartier
    pass
  
  try :
    df_final.loc[df_final.quartier == quartier, "nb_chambre_hotel"] = types_compteur[quartier]["Hotel room"]
  except :
    #Cas où il n'existe pas d'annonce correspondant au type de logement pour ce quartier
    pass
  
  try :
    df_final.loc[df_final.quartier == quartier, "nb_chambre_ptg"] = types_compteur[quartier]["Shared room"]
  except :
    #Cas où il n'existe pas d'annonce correspondant au type de logement pour ce quartier
    pass

In [78]:
#Calcul de la capacité maximal moyenne par quartier
df_final["occup_max_moy"] = df_final.groupby("quartier")["occupation_max"].transform("mean")

In [79]:
#Aperçu
df_final.head()

,id,quartier,type_chambre,occupation_max,hotes_incluses,prix,frais_nettoyage,prix_total_pp,prix_moyen,prix_min,prix_max,prix_med,nb_annonces,nb_log_entier,nb_chambre_privee,nb_chambre_hotel,nb_chambre_ptg,occup_max_moy
0,21974,el Barri Gòtic,Private room,2,2,80.0,20.0,50.0,101.620674,13.500000,5060.0,64.000000,761,205,552,3,1,3.123522
1,23197,el Besòs i el Maresme,Entire home/apt,6,3,220.0,80.0,100.0,91.461574,17.777778,1475.0,46.250000,96,36,60,0,0,3.218750
2,34241,el Barri Gòtic,Private room,2,1,100.0,40.0,140.0,101.620674,13.500000,5060.0,64.000000,761,205,552,3,1,3.123522
3,34981,el Barri Gòtic,Entire home/apt,9,4,227.0,67.0,73.5,101.620674,13.500000,5060.0,64.000000,761,205,552,3,1,3.123522
4,35379,l'Antiga Esquerra de l'Eixample,Private room,2,1,40.0,6.0,46.0,149.978822,5.000000,6000.0,60.333333,752,391,320,32,9,4.037234


In [80]:
#Nbre de lignes/colonnes du fichier final.
df_final.shape

(12462, 18)

In [81]:
#Sélection des colonnes à conserver, suppression des doublons pour obtenir les données par quartier
df_export = df_final[["quartier", "prix_moyen", "prix_min", "prix_max", "prix_med", "nb_annonces", "nb_log_entier", "nb_chambre_privee", "nb_chambre_hotel", "nb_chambre_ptg", "occup_max_moy"]].drop_duplicates().sort_values(by = ["quartier"], ignore_index = True)

In [82]:
#Conversion de la première lettre du nom du quartier en majuscule
df_export["quartier"] = df_export["quartier"].apply(lambda x : x[0].upper() +  x[1:])

In [83]:
#Sélection des quartiers pour lesquelles au moins 100 annonces existent
df_export = df_export[df_export["nb_annonces"] >= 100].reset_index(drop = True)

In [84]:
#Tri par ordre alphabétique du quartier
df_export = df_export.sort_values(by = ["quartier"])

In [85]:
#Tableau final
df_export

,quartier,prix_moyen,prix_min,prix_max,prix_med,nb_annonces,nb_log_entier,nb_chambre_privee,nb_chambre_hotel,nb_chambre_ptg,occup_max_moy
0,Diagonal Mar i el Front Marítim del Poblenou,206.259412,20.000000,1780.000000,80.000000,139,95,36,8,0,5.187050
7,El Barri Gòtic,101.620674,13.500000,5060.000000,64.000000,761,205,552,3,1,3.123522
8,El Camp d'en Grassot i Gràcia Nova,71.183497,16.500000,400.000000,52.500000,204,115,81,8,0,3.705882
9,El Camp de l'Arpa del Clot,90.047003,14.000000,1722.000000,64.500000,205,113,92,0,0,3.692683
10,El Fort Pienc,94.051116,15.000000,999.000000,63.333333,369,196,165,6,2,4.046070
11,El Guinardó,58.870219,10.500000,650.000000,45.000000,122,33,88,0,1,3.032787
12,El Parc i la Llacuna del Poblenou,77.443197,17.500000,845.833333,58.750000,147,67,76,3,1,3.401361
13,El Poble Sec,92.664500,10.000000,2535.000000,57.000000,766,454,291,8,13,4.026110
14,El Poblenou,209.449247,16.500000,5060.000000,73.000000,332,175,144,10,3,3.545181
15,El Putxet i el Farró,74.991354,16.285714,290.000000,60.000000,141,92,49,0,0,3.865248


In [86]:
#Export en .csv
df_export.to_csv("donnees_quartiers.csv", index=False)